# Analysis Sample

In [1]:
from pathlib import Path
import PIL
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
# Cancer pathology samples
data_dir = "./data/"
img_paths = {
    path.stem: path for path in Path(data_dir).glob("*.TIF")
}

print(img_paths)

images = {
    name: np.array(PIL.Image.open(path))
    for name, path in img_paths.items()
}

{'Dual fusion': WindowsPath('data/Dual fusion.TIF'), 'separation probe': WindowsPath('data/separation probe.TIF'), 'Trisomy 8': WindowsPath('data/Trisomy 8.TIF')}


In [5]:
from ufish.api import UFish
from cellpose import models

# initialize cellpose model
cp = models.Cellpose(gpu=False, model_type="nuclei")

# initialize ufish model
uf = UFish(device="cuda")
uf.load_weights("./v1.0.1-DNAFISH_model.onnx")
# weights link: https://huggingface.co/GangCaoLab/U-FISH/blob/main/finetune_models/v1.0.1-DNAFISH_model.pth

from utils import pipeline, plot_figs, plot_all_rois

2024-06-17 13:29:29.336 | INFO     | ufish.api:_load_onnx:296 - Loading ONNX from ./v1.0.1-DNAFISH_model.onnx
2024-06-17 13:29:29.337 | INFO     | ufish.api:_load_onnx:303 - Using ONNX runtime providers: ['CUDAExecutionProvider']


In [6]:
from shutil import rmtree
import os
main_dir = "results"
os.makedirs(main_dir, exist_ok=True)
for name, img in images.items():
    print(name)
    res_dir = f"./{main_dir}/{name}"
    if os.path.exists(res_dir):
        rmtree(f"./{res_dir}")
    os.mkdir(f"./{res_dir}")

    if name.startswith("Trisomy"):
        signal_channels=[0]
    else:
        signal_channels=[0, 1]
    table, rois, masks, signals= pipeline(cp, uf, img, signal_channels=signal_channels)

    plot_figs(rois, masks, signals, res_dir)

    fig = plot_all_rois(rois, masks, signals, colors={"ch1": "hotpink", "ch2": "lime", "ch1+ch2": "yellow"})
    fig.savefig(f"{res_dir}/{name}_all_cells.pdf")
    plt.close(fig)

    table.to_csv(f"{res_dir}/{name}_results.csv", index=False)

Dual fusion
Processing image:
Image shape: (3000, 4096, 3)
Step 1: segment cells
Step 2: extract ROIs
Step 3: call spots and assign spots


2024-06-17 13:32:21.738 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:32:21.738 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:32:26.524 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:32:26.525 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:32:26.555 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:32:26.556 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:32:26.566 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:32:26.568 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:32:26.597 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:32:26.598 | INFO     | ufish.api:p

separation probe
Processing image:
Image shape: (3000, 4096, 3)
Step 1: segment cells


2024-06-17 13:34:53.419 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:34:53.419 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.453 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:34:53.455 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.494 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.


Step 2: extract ROIs
Step 3: call spots and assign spots


2024-06-17 13:34:53.496 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.517 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:34:53.518 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.556 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:34:53.556 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.573 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:34:53.573 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.612 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:34:53.613 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:34:53.632 | INFO     | ufish.api:predic

Trisomy 8
Processing image:
Image shape: (3000, 4096, 3)
Step 1: segment cells


2024-06-17 13:37:08.079 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:37:08.080 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:37:08.104 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.


Step 2: extract ROIs
Step 3: call spots and assign spots


2024-06-17 13:37:08.105 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:37:08.130 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:37:08.132 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:37:08.157 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:37:08.158 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:37:08.182 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:37:08.184 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:37:08.207 | INFO     | ufish.api:predict:469 - Axes not specified, infering from image shape.
2024-06-17 13:37:08.207 | INFO     | ufish.api:predict:471 - Infered axes: yx, image shape: (128, 128)
2024-06-17 13:37:08.233 | INFO     | ufish.api:predic